In [7]:
import os
import pandas as pd
import numpy as np

from reportlab.pdfgen import canvas
from reportlab.lib import colors, utils
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

print('Pandas version:',pd.__version__)
print('Numpy version:', np.__version__)
print('Reportlab version:', canvas.__version__)

Pandas version: 2.2.1
Numpy version: 1.26.4
Reportlab version: 3.3.0


In [2]:
#Get current and parent directory
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)

#Path to the source folder where the raw data is located
source_path = os.path.join(parent_directory, 'data', 'raw', 'data_per_day.xlsx')

#Path to the image
image_path = os.path.join(parent_directory, 'graphics', 'bow.png')

#Read the .xlsx file
df = pd.read_excel(source_path, index_col=None)

In [3]:
#Main distinction is saved in the system as 'NA' and 'NNA'. 'NA' is read as a empty value.
#We need fill 'empty' value as a teachers, and change 'NNA' as a not teachers.
df['Grupa pracowników'] = df['Grupa pracowników'].fillna('Nauczyciel')
df['Grupa pracowników'] = df['Grupa pracowników'].replace('NNA', 'Nienauczyciel')

In [4]:
#Style for table register
styles = getSampleStyleSheet()
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#66CC00')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Calibri-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 10),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),
    ('BACKGROUND', (0, 1), (-1, -1), colors.white),
    ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
    ('ALIGN', (0, 1), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 1), (-1, -1), 'Calibri'),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 2),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
])

#Definition to convert df into table and location in pdf
def add_df_too_pdf(name, x, y):
    header_row = name.columns.to_list()
    table_data = [header_row] + name.values.tolist()

    pdf_table = Table(table_data, style=table_style)

    pdf_table.wrapOn(c, 400, 600)  # 400 and 600 is width and height of the area in which we draw the table
    pdf_table.drawOn(c, x, y)  # x and y is these are the coordinates where we draw the table on the page

In [5]:
if __name__ == "__main__":
    
    #Columns over which the data will be iterated
    selected_columns = ['Pion', 'Obszar kadrowy']

    for col in selected_columns:
        for val in df[col].unique():

            #Path to the destination file
            destination_path = os.path.join(parent_directory, 'data', 'processed', col + '_' + val + '.pdf')

            #Create pdf file and font register
            c = canvas.Canvas(destination_path, pagesize=letter)
            pdfmetrics.registerFont(TTFont('Calibri', 'calibri.ttf'))
            pdfmetrics.registerFont(TTFont('Calibri-Bold', 'calibrib.ttf'))
            c.setFont('Calibri', 20)

            #Add name
            c.drawString(235, 720, "HR - employee stats")

            #Add image
            img = utils.ImageReader(image_path)
            c.drawImage(img, 205, 715, 25, 25)

            #Create a pivot table based on the unique values in the selected column
            df_pivot_mainGroups = df[df[col] == val].pivot_table(values='Etat', index='Grupa pracowników', columns=col, 
                                  aggfunc=['count', 'sum']).applymap(lambda x: round(x, 2))
            
            #Reset the index to make 'Grupa pracowników' a regular column
            df_pivot_mainGroups = df_pivot_mainGroups.reset_index() 
            
            #Rename the columns for clarity
            df_pivot_mainGroups.columns = ['Grupa', 'Osoby', 'Etaty'] 
            
            #Set the coordinates for placing the table in the PDF
            x=350 
            y=620
            
            #Add the table to the PDF
            add_df_too_pdf(df_pivot_mainGroups, x, y)

            
            #comments as with the first pivot
            df_pivot_teachersJobPosition = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                           'Etat', index='Stanowisko', columns=col, aggfunc=['count', 'sum']) \
                                           .applymap(lambda x: round(x, 2))
            df_pivot_teachersJobPosition = df_pivot_teachersJobPosition.reset_index()
            df_pivot_teachersJobPosition.columns = ['NA Stanowiska', 'Osoby', 'Etaty']
            x=90
            y=552
            add_df_too_pdf(df_pivot_teachersJobPosition, x, y)


            
            df_pivot_teachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                        'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                        .applymap(lambda x: round(x, 2))
            df_pivot_teachersSubgroup.columns = df_pivot_teachersSubgroup.columns.droplevel(1)
            df_pivot_teachersSubgroup = df_pivot_teachersSubgroup.reset_index()
            df_pivot_teachersSubgroup.columns = ['NA Podgrupa', 'Osoby', 'Etaty']
            x=90
            y=455
            add_df_too_pdf(df_pivot_teachersSubgroup, x, y)


            
            df_pivot_notteachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                           'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                           .applymap(lambda x: round(x, 2))
            df_pivot_notteachersSubgroup = df_pivot_notteachersSubgroup.reset_index()
            df_pivot_notteachersSubgroup.columns = ['NNA Grupa', 'Osoby', 'Etaty']
            x=350
            y=435
            add_df_too_pdf(df_pivot_notteachersSubgroup, x, y)


            
            df_teachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                           'Wynagrodzenie', index=['Podgrupa pracowników', 'Stanowisko'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                           .applymap(lambda x: round(x, 2))
            df_teachersSalary = df_teachersSalary.reset_index()
            df_teachersSalary.columns = ['NA Podgrupa','Stanowisko', 'Osoby', 'Średnia', 'Mediana']
            df_teachersSalary[['Średnia', 'Mediana']] = df_teachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
            df_teachersSalary = df_teachersSalary.sort_values(by=['Stanowisko'])
            x=30
            y=170
            add_df_too_pdf(df_teachersSalary, x, y)


            
            df_notteachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                           'Wynagrodzenie', index=['Podgrupa pracowników'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                           .applymap(lambda x: round(x, 2))
            df_notteachersSalary = df_notteachersSalary.reset_index()
            df_notteachersSalary.columns = ['NNA Podgrupa', 'Osoby', 'Średnia', 'Mediana']
            df_notteachersSalary[['Średnia', 'Mediana']] = df_notteachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
            x=350
            y=250
            add_df_too_pdf(df_notteachersSalary, x, y)


            # Save pdf
            c.save()

            
            print(f'Successfully create pdf - {col}_{val}')

Successfully create pdf - Pion_Rektor
Successfully create pdf - Pion_Rektor_2
Successfully create pdf - Obszar kadrowy_Wydział 1
Successfully create pdf - Obszar kadrowy_Wydział 10
Successfully create pdf - Obszar kadrowy_Wydział 2
Successfully create pdf - Obszar kadrowy_Wydział 3
Successfully create pdf - Obszar kadrowy_Wydział 4
Successfully create pdf - Obszar kadrowy_Wydział 5
Successfully create pdf - Obszar kadrowy_Wydział 6
Successfully create pdf - Obszar kadrowy_Wydział 7
Successfully create pdf - Obszar kadrowy_Wydział 8
Successfully create pdf - Obszar kadrowy_Wydział 9
